In [ ]:
import librosa
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import torch
from qdrant_client.http.models import PointStruct
audio_path="A:\Projects\Audio-Rag\Data\wav_audio\sample-3s.wav"



In [64]:
def get_audio_in_number(audio_path:str):
    audio, sr = librosa.load(audio_path, sr=16000)  # load audio at 16kHz
    return audio
def get_processor_model():
    """
    returns: model, processor
    """
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
    return model, processor

def get_embedding(audio):
    model, processor = get_processor_model()
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
    with torch.no_grad():
        embeddings = model(input_values).last_hidden_state  # (batch_size, sequence_length, hidden_size)
    audio_embedding = embeddings.mean(dim=1)  # (batch_size, hidden_size)
    return audio_embedding[0] 

def insert_in_collection(collection_name:str,embedding,path):
    operation_info = client.upsert(
        collection_name="audio_vectors",
        wait=True,
        points=[
            PointStruct(id=1, vector=audio_embedding, payload={"city": audio_path}),
        ]
    )

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

# Connect to your Qdrant server (local or cloud)
client = QdrantClient(path="A:\Projects\Audio-Rag\qdrant")

# Create a collection
client.recreate_collection(
    collection_name="audio_vectors",
    vectors_config=VectorParams(
        size=768,                   # size of your vector (example: 512 dims)
        distance=Distance.COSINE,    # distance metric: COSINE / EUCLID / DOT
        on_disk=True                 # important: store vectors on disk
    )
)

C:\Users\vivek\AppData\Local\Temp\ipykernel_17548\4214607031.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [65]:
# Get audio as a numpy array (or however your get_audio_in_number works)
audio = get_audio_in_number(audio_path)

# Get the embedding
query_vector = get_embedding(audio)

# Convert to list for Qdrant
query_vector = query_vector.tolist()  # No need to squeeze again, already shape (hidden_size,)

# Now search
search_result = client.search(
    collection_name="audio_vectors",
    query_vector=query_vector,  # Correct variable
    limit=1
)

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\vivek\AppData\Local\Temp\ipykernel_17548\2886985831.py:11: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


TypeError: only integer scalar arrays can be converted to a scalar index